In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
import time 
import smtplib
from dotenv import load_dotenv   #for python-dotenv method
import os

In [2]:
env_path = os.getcwd() + '\\creds.env'
load_dotenv(dotenv_path=env_path)

True

In [3]:
# email address and pw of the program
# change as needed
gmail_user = os.environ.get('gmail_user')
gmail_password = os.environ.get('gmail_password')

# the email addresses that you'd like to notify
recipients = ['someemail@gmail.com']

# your cwl and pw
cwl = os.environ.get('cwl')
pw = os.environ.get('pw')
wait_time = 30
# something that looks like 'https://courses.students.ubc.ca/cs/courseschedule?pname=subjarea&tname=subj-section&dept=ECON&course=102&section=001' ie a webpage with the option of clicking register
course_url = 'https://courses.students.ubc.ca/cs/courseschedule?pname=subjarea&tname=subj-section&dept=MATH&course=419&section=201'

In [4]:
# whether to send email notification of registration success
notify = True

In [5]:
def notify_me(txt):
    sent_from = gmail_user
    to = recipients
    subject = 'Course registration ' + txt
    body = course_url

    email_text = """\
From: %s
To: %s
Subject: %s

%s
""" % (sent_from, ", ".join(to), subject, body)
    print(email_text)

    try:
        smtp_server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
        smtp_server.ehlo()
        smtp_server.login(gmail_user, gmail_password)
        smtp_server.sendmail(sent_from, to, email_text)
        smtp_server.close()
        print ("Email sent successfully!")
    except Exception as ex:
        print ("Something went wrong….",ex)

In [6]:
# see https://stackoverflow.com/questions/42478591/python-selenium-chrome-webdriver if running into trouble
# update path to chromedriver
chromedriver = Service('C:\\chromedriver.exe')
browser = webdriver.Chrome(service = chromedriver, options = webdriver.ChromeOptions())
browser.get(course_url)

In [9]:
# login cwl, must do this if have two factor auth
def login_cwl():
    browser.switch_to.window(browser.window_handles[0])
    browser.execute_script('''window.open("https://cas.id.ubc.ca/ubc-cas/login?TARGET=https%3A%2F%2Fcourses.students.ubc.ca%2Fcs%2Fsecure%2Flogin%3FIMGSUBMIT.x%3D72%26IMGSUBMIT.y%3D15","_blank");''')
    browser.switch_to.window(browser.window_handles[-1])
    cwlfield = browser.find_element(By.ID, 'username')
    pwfield = browser.find_element(By.ID, 'password')
    cwlfield.clear()
    cwlfield.send_keys(cwl)
    pwfield.clear()
    pwfield.send_keys(pw)
    browser.find_element(By.NAME, 'submit').click()

    
# goes to specified url for the course and clicks register section
def register_course():
    try:
        login_cwl()
    except Exception as ex:
        print('cwl probably logged in')
    time.sleep(5)
    # replace link with course_url here
    browser.execute_script('''window.open('https://courses.students.ubc.ca/cs/courseschedule?pname=subjarea&tname=subj-section&dept=MATH&course=419&section=201',"_blank");''')
    browser.close()
    browser.switch_to.window(browser.window_handles[-1])
    browser.find_element(By.XPATH, '/html/body/div[2]/div[4]/a[2]').click()
    
    
# check if registration is successful
def check_status():
    msg = browser.find_element(By.XPATH, '/html/body/div[2]/div[4]/div[2]/b').text
    browser.close()
    browser.switch_to.window(browser.window_handles[0])
    print(msg)
    if 'Unsuccessful' in msg:
        return False
    return True


# returns the number of general seats remaining
def find_seat():
    seat_name = browser.find_element(By.XPATH, '/html/body/div[2]/div[4]/table[4]/tbody/tr[3]/td[1]').text
    # checks that we are indeed checking for general seats and not something random
    assert(seat_name == 'General Seats Remaining:')
    table_id = browser.find_element(By.XPATH, '/html/body/div[2]/div[4]/table[4]/tbody/tr[3]/td[2]')
    return int(table_id.text)


# idk how to fix this, not used
def logout_cwl():
    browser.find_element(By.ID, 'cwl-logout').click()
    driver.close()
    browser.switch_to.window(browser.window_handles[0])

In [ ]:
find_seat()

In [261]:
#RUN THIS FIRST BEFORE RUNNING THE FOR LOOP
login_cwl()
# close the tab after you are logged in

In [ ]:
import time 
while True:
    time.sleep(wait_time)
    browser.switch_to.window(browser.window_handles[0])
    browser.refresh();
    try:
        num_seats = find_seat()
        if num_seats > 0:
            notify_me('avaliable')
            register_course()
            if check_status():
                if notify:
                    notify_me('success')
                break
            else: 
                if notify:
                    notify_me('failed')
    except Exception as ex:
        print('sum ting wong')

In [81]:
browser.quit()